In [1]:
import h3 
import xarray as xr
import rasterio
from shapely.geometry import Polygon
import geopandas as gpd

Se leen los datos de velocidad del viento que contienen toda la extension del proyecto. Se busca utilizar la extensión del mismo para definir la región a llenar con hexágonos.

In [3]:
# wind_speed
path = '/home/jta/Documentos/raster_variables/ahpAmbiental/COL_wind-speed_10m.tif'
ds_wind = rasterio.open(path)

Para generar una malla de hexagonos se genera un poligono que contiene los limites del raster.

In [18]:
ext = ds_wind.bounds
up_l_corner = (ext[-1],ext[0])
up_r_corner = (ext[-1],ext[2])
down_l_corner = (ext[1],ext[0])
down_r_cornor = (ext[1],ext[2])
# Creacion poligono
polygon = Polygon(
    (
        down_l_corner,down_r_cornor,
        up_r_corner,up_l_corner
    )
)


la funcion `geo_to_cells` genera la malla de hexagonos, a esta se le debe suministrar la resolucion con el tamano de los hexagonos. Esta regresa una lista de `str` que contienen el codigo de cada hexagonos

In [22]:
# Creacion de una malla de hexagonos
resolution = 8
hexagons = h3.geo_to_cells(polygon, resolution)


De cada hexagono se pueden extraer los vertices de cada hexagono, esto se puede convertir en un geodataframe para almacenar la informacion asociada a cada variable. Donde cada columna del geodataframe corresponde a un atributo especifico

In [26]:
# Genera lista con la extension de 
geometry = [Polygon(h3.cell_to_boundary(h)) for h in hexagons]

gdf_hex = gpd.GeoDataFrame(
    {'h3': hexagons},
    geometry=geometry,
    crs="EPSG:4326"
)
# cambio de epsg
epsg_on = '9377'
gdf_hex = gdf_hex.to_crs(epsg_on)

In [27]:
# Lectura de datos
path = '/home/jta/Documentos/raster_variables/datos_cris/especies_densidades/especies_migratorias_final'
gdf_migratorias = gpd.read_file(path)
# convertir 
gdf_migratorias = gdf_migratorias.to_crs(epsg_on)

In [41]:
def densidad_especies(
        h3_hex:gpd.GeoDataFrame,
        gdf_especies:gpd.GeoDataFrame
    ) -> gpd.GeoDataFrame:

    if h3_hex.crs != gdf_migratorias.crs:
        gdf_especies = gdf_especies.to_crs(h3_hex.crs)

    # 2. Realizar el Spatial Join
    # Esto asigna a cada punto el ID del hexágono donde se encuentra
    # 'predicate="within"' es lo inverso a contains, ideal para puntos dentro de polis
    puntos_con_hex = gpd.sjoin(gdf_especies, h3_hex, how="inner", predicate="within")

    # 3. Contar cuántos puntos hay por cada índice de hexágono
    # Asumimos que el índice de h3_hex es único (ej. el h3 address)
    conteo_puntos = puntos_con_hex.groupby("index_right").size()

    # 4. Asignar el conteo al geodataframe original de hexágonos
    h3_hex.loc[:,"num_avistamientos"] = conteo_puntos
    h3_hex.loc[:,"num_avistamientos"] = h3_hex["num_avistamientos"].fillna(0) # Los que no tuvieron puntos son 0

    # 5. Calcular densidad (Vectorizado - sin bucles)
    #Area en km2 (asumiendo que el CRS proyectado está en metros, si es geográfico lat/lon es diferente)
    factor = 1e6
    h3_hex.loc[:,"area_km2"] = h3_hex.geometry.area / factor
    h3_hex.loc[:,"densidad"] = h3_hex["num_avistamientos"] / h3_hex["area_km2"]
    
    return h3_hex

In [42]:
h3_especies = densidad_especies(gdf_hex,gdf_migratorias)

Manejo de ráster con H3

In [50]:
path_pendientes = '/home/jta/Documentos/raster_variables/raster_data/Pendientes.tif'
ds = xr.open_dataset(path_pendientes)

In [69]:
prueba = gdf_hex['geometry'].iloc[0]
ds['band_data'].rio.clip([prueba]).values.mean()

np.float32(0.0036275585)

In [81]:
def raster_2_h3(
        h3:gpd.GeoDataFrame,
        raster:xr.DataArray,
    ) -> gpd.GeoDataFrame:

    clip_values = raster.rio.clip([h3])
    mean_val = clip_values.values.mean()
    # return mean value
    return mean_val

In [82]:
da = ds['band_data']
gdf_hex['pendiente'] = gdf_hex['geometry'].apply(raster_2_h3,args=(da,))

NoDataInBounds: No data found in bounds. Data variable: band_data

In [80]:
gdf_hex

,h3,geometry,num_avistamientos,area_km2,densidad
0,88e74ab153fffff,"POLYGON ((5325672.764 2270751.835, 5326192.707...",0.0,2.390315,0.0
1,88e74e391bfffff,"POLYGON ((5277632.122 2453118.543, 5278155.352...",0.0,2.425161,0.0
2,88ef9e3307fffff,"POLYGON ((4714660.425 1703928.869, 4715184.234...",0.0,3.421767,0.0
3,88e75c583bfffff,"POLYGON ((5400394.434 3191045.991, 5400910.973...",0.0,2.168994,0.0
4,88f1693865fffff,"POLYGON ((3845240.623 1229251.959, 3845730.612...",0.0,7.507702,0.0
...,...,...,...,...,...
1305221,88e758b033fffff,"POLYGON ((5428545.561 2785409.567, 5429063.776...",0.0,2.206401,0.0
1305222,88eeb31139fffff,"POLYGON ((5181829.665 1182933.908, 5182326.141...",0.0,2.649389,0.0
1305223,88ef9e12c7fffff,"POLYGON ((4690862.544 1837828.981, 4691391.747...",0.0,3.464347,0.0
1305224,88f1691825fffff,"POLYGON ((3797756.295 1251269.183, 3798246.824...",0.0,8.010485,0.0
